In [ ]:
from paz.backend.image import load_image, show_image, write_image
from paz.applications import HaarCascadeFrontalFace, MiniXceptionFER
import paz.processors as pr
from paz.backend.image.image import crop_image
from paz.backend.image import write_image
from paz.processors.detection import CropBoxes2D
import numpy as np
from pathlib import Path
from paz.abstract import SequentialProcessor
from retinaface import RetinaFace
from paz.abstract.messages import Box2D
from paz.backend import boxes
from paz.backend.image import opencv_image
import os
import csv

#create an empty file emo_list.csv

csvfile = open('emo_list.csv', 'x')
csvfile.close()
front_face_size = np.array([224, 224])
dict = {   
    "neutral": 0,
    "happy": 1,
    "sad": 2,
    "surprise": 3,
    "fear": 4,
    "disgust": 5,
    "angry": 6,
    "contempt": 7
}  

#loop through each of the c1 ... c9 folder
for distraction_num in range(0, 10):
    SideBody_folder_str = './datasets/DMD/SideBody/train/' + 'c%(number)01d/' % {"number": distraction_num}
    
    #loop through every sidebody frame in the folder cn   
    for file_name in os.listdir(SideBody_folder_str):
        if file_name.endswith(".png"):   #framename format: gxxsxsidebody_000xxx.png
            SideBody_frame_name = file_name
            SideBody_path = SideBody_folder_str + file_name
            
            #find its corresponding frontbody frame
            FrontBody_folder_str = './datasets/DMD/FrontBody/'
            FrontBody_frame_name = SideBody_frame_name[0:5] + 'front' + SideBody_frame_name[9:]
            FrontBody_path = FrontBody_folder_str + FrontBody_frame_name
            FrontBody_image = load_image(FrontBody_path)
            
            #Detect and crop the front face
            crop = CropBoxes2D()
            face = RetinaFace.detect_faces(FrontBody_path)
            coordinates = face['face_1']['facial_area']
            boxes2D = [Box2D(coordinates, score=0)]
            
            #edge case 1: PAZ fails to detect a face
            if (len(boxes2D) == 0):
                result = np.append(result, [[SideBody_path, 'noFaceDetected']], axis=0) # how to cover this case in test?
                print('No face detected for this frame')
                continue
        
            #edge case 2: PAZ detects multiple "faces" - some of these are non-face areas
            elif (len(boxes2D) >= 2):
                areas = np.array([])
                for box in boxes2D:
                    area = (box.coordinates[2]-box.coordinates[0]) * (box.coordinates[3]-box.coordinates[1])
                    areas = np.append(areas, area)
                indices_excluding_max = np.where(areas != np.amax(areas))[0]
                for i in indices_excluding_max:
                    boxes2D = np.delete(boxes2D, i)
                print('After removing smaller boxes: ', boxes2D) 
            cropped_images = crop(FrontBody_image, boxes2D)
            
            #Label its emotion
            #According to the 7 class labels of the MiniXeptionFER model, 
            #there should not be any classes mapping to -1 or 7.
            classify = MiniXceptionFER()
            emotion_str = classify(cropped_images[0])['class_name']
            if (emotion_str in dict.keys()):
                emotion = dict.get(emotion_str)
            else:
                emotion = -1
            

            #Add a row to emo_list.csv [original SideBody frame path, emotion]
            with open('emo_list.csv', 'a') as csvfile:
                csvwriter = csv.writer(csvfile, lineterminator = '\n') 
                csvwriter.writerow([SideBody_path, emotion])
            print(SideBody_frame_name, ': ', [SideBody_path, emotion, emotion_str])
            
            #Resize the cropped face to 224*224
            if (cropped_images[0].shape[0] != cropped_images[0].shape[1]):
                new_coordinates = boxes.make_box_square(boxes2D[0].coordinates)
                box2D = Box2D(new_coordinates, 0)
                boxes2D = [box2D]
                cropped_images = crop(FrontBody_image, boxes2D)
            resized_image = opencv_image.resize_image(cropped_images[0], front_face_size)
                    
            #create directories to store the cropped faces
            os.makedirs('./pseudo_emotion_label/DMD/imgs/', exist_ok=True)
            
            #Save the cropped front face images
            cropped_image_name = FrontBody_frame_name[0:5] + 'frontcropped' + FrontBody_frame_name[14:]
            save_path = './pseudo_emotion_label/DMD/imgs/' + cropped_image_name
            write_image(save_path, resized_image)
            
        #skip non-png files/directory, if any        
            else:
            continue



Updated code on 03/16 evening: Fix error catching on handling boxes2D

In [1]:
from paz.backend.image import load_image, show_image, write_image
from paz.applications import HaarCascadeFrontalFace, MiniXceptionFER
import paz.processors as pr
from paz.backend.image.image import crop_image
from paz.backend.image import write_image
from paz.processors.detection import CropBoxes2D
import numpy as np
from pathlib import Path
from paz.abstract import SequentialProcessor
from retinaface import RetinaFace
from paz.abstract.messages import Box2D
from paz.backend import boxes
from paz.backend.image import opencv_image
import os
import csv

#create an empty file emo_list.csv

csvfile = open('emo_list.csv', 'x')
csvfile.close()
front_face_size = np.array([224, 224])
dict = {   
    "neutral": 0,
    "happy": 1,
    "sad": 2,
    "surprise": 3,
    "fear": 4,
    "disgust": 5,
    "angry": 6,
    "contempt": 7
}  

#loop through each of the c1 ... c9 folder
for distraction_num in range(0, 10):
    SideBody_folder_str = './datasets/DMD/SideBody/train/' + 'c%(number)01d/' % {"number": distraction_num}
    
    #loop through every sidebody frame in the folder cn   
    for file_name in os.listdir(SideBody_folder_str):
        if file_name.endswith(".png"):   #framename format: gxxsxsidebody_000xxx.png
            SideBody_frame_name = file_name
            SideBody_path = SideBody_folder_str + SideBody_frame_name
            
            #find its corresponding frontbody frame
            FrontBody_folder_str = './datasets/DMD/FrontBody/'
            FrontBody_frame_name = SideBody_frame_name[0:5] + 'front' + SideBody_frame_name[9:]
            FrontBody_path = FrontBody_folder_str + FrontBody_frame_name
            FrontBody_image = load_image(FrontBody_path)

            #Detect and crop the front face
            crop = CropBoxes2D()
            face = RetinaFace.detect_faces(FrontBody_path)
            try:
                coordinates = face['face_1']['facial_area']
                boxes2D = [Box2D(coordinates, score=0)]
            except Exception: 
                boxes2D = []


            #edge case 1: PAZ fails to detect a face
            if (len(boxes2D) == 0):
                print('No face detected for this frame, no row added to emo_list')
                continue


            #edge case 2: PAZ detects multiple "faces" - some of these are non-face areas
            elif (len(boxes2D) >= 2):
                areas = np.array([])
                for box in boxes2D:
                    area = (box.coordinates[2]-box.coordinates[0]) * (box.coordinates[3]-box.coordinates[1])
                    areas = np.append(areas, area)
                indices_excluding_max = np.where(areas != np.amax(areas))[0]
                for i in indices_excluding_max:
                    boxes2D = np.delete(boxes2D, i)
                print('After removing smaller boxes: ', boxes2D) 
            cropped_images = crop(FrontBody_image, boxes2D)
            
            #Label its emotion
            #According to the 7 class labels of the MiniXeptionFER model, 
            #there should not be any classes mapping to -1 or 7.
            classify = MiniXceptionFER()
            emotion_str = classify(cropped_images[0])['class_name']
            if (emotion_str in dict.keys()):
                emotion = dict.get(emotion_str)
            else:
                emotion = -1
            

            #Add a row to emo_list.csv [original SideBody frame path, emotion]
            with open('emo_list.csv', 'a') as csvfile:
                csvwriter = csv.writer(csvfile, lineterminator = '\n') 
                csvwriter.writerow([SideBody_path, emotion])
            print(SideBody_frame_name, ': ', [SideBody_path, emotion, emotion_str])
            
            #Resize the cropped face to 224*224
            if (cropped_images[0].shape[0] != cropped_images[0].shape[1]):
                new_coordinates = boxes.make_box_square(boxes2D[0].coordinates)
                box2D = Box2D(new_coordinates, 0)
                boxes2D = [box2D]
                cropped_images = crop(FrontBody_image, boxes2D)
            resized_image = opencv_image.resize_image(cropped_images[0], front_face_size)
                    
            #create directories to store the cropped faces
            os.makedirs('./pseudo_emotion_label/DMD/imgs/', exist_ok=True)
            
            #Save the cropped front face images
            cropped_image_name = FrontBody_frame_name[0:5] + 'frontcropped' + FrontBody_frame_name[14:]
            save_path = './pseudo_emotion_label/DMD/imgs/' + cropped_image_name
            write_image(save_path, resized_image)
            
        #skip non-png files/directory, if any        
        else:
            continue

ga1s1sidebody_000002.png :  ['./datasets/DMD/SideBody/train/c0/ga1s1sidebody_000002.png', 0, 'neutral']
ga1s1sidebody_000003.png :  ['./datasets/DMD/SideBody/train/c0/ga1s1sidebody_000003.png', 0, 'neutral']
ga1s1sidebody_000004.png :  ['./datasets/DMD/SideBody/train/c0/ga1s1sidebody_000004.png', 0, 'neutral']
ga1s1sidebody_000005.png :  ['./datasets/DMD/SideBody/train/c0/ga1s1sidebody_000005.png', 0, 'neutral']
ga1s1sidebody_000006.png :  ['./datasets/DMD/SideBody/train/c0/ga1s1sidebody_000006.png', 0, 'neutral']
ga1s1sidebody_000007.png :  ['./datasets/DMD/SideBody/train/c0/ga1s1sidebody_000007.png', 2, 'sad']
ga1s1sidebody_000008.png :  ['./datasets/DMD/SideBody/train/c0/ga1s1sidebody_000008.png', 2, 'sad']
ga1s1sidebody_000009.png :  ['./datasets/DMD/SideBody/train/c0/ga1s1sidebody_000009.png', 2, 'sad']
ga1s1sidebody_000010.png :  ['./datasets/DMD/SideBody/train/c0/ga1s1sidebody_000010.png', 2, 'sad']
ga1s1sidebody_000011.png :  ['./datasets/DMD/SideBody/train/c0/ga1s1sidebody_000

ga1s1sidebody_000166.png :  ['./datasets/DMD/SideBody/train/c0/ga1s1sidebody_000166.png', 0, 'neutral']
ga1s1sidebody_000167.png :  ['./datasets/DMD/SideBody/train/c0/ga1s1sidebody_000167.png', 2, 'sad']
ga1s1sidebody_000171.png :  ['./datasets/DMD/SideBody/train/c0/ga1s1sidebody_000171.png', 1, 'happy']
ga1s1sidebody_000173.png :  ['./datasets/DMD/SideBody/train/c0/ga1s1sidebody_000173.png', 1, 'happy']
ga1s1sidebody_000174.png :  ['./datasets/DMD/SideBody/train/c0/ga1s1sidebody_000174.png', 1, 'happy']
ga1s1sidebody_000176.png :  ['./datasets/DMD/SideBody/train/c0/ga1s1sidebody_000176.png', 1, 'happy']
ga1s1sidebody_000178.png :  ['./datasets/DMD/SideBody/train/c0/ga1s1sidebody_000178.png', 1, 'happy']
ga1s1sidebody_000180.png :  ['./datasets/DMD/SideBody/train/c0/ga1s1sidebody_000180.png', 1, 'happy']
ga1s1sidebody_000181.png :  ['./datasets/DMD/SideBody/train/c0/ga1s1sidebody_000181.png', 1, 'happy']
ga1s1sidebody_000182.png :  ['./datasets/DMD/SideBody/train/c0/ga1s1sidebody_00018

ga1s2sidebody_000126.png :  ['./datasets/DMD/SideBody/train/c0/ga1s2sidebody_000126.png', 2, 'sad']
ga1s2sidebody_000127.png :  ['./datasets/DMD/SideBody/train/c0/ga1s2sidebody_000127.png', 0, 'neutral']
ga1s2sidebody_000128.png :  ['./datasets/DMD/SideBody/train/c0/ga1s2sidebody_000128.png', 0, 'neutral']
ga1s2sidebody_000129.png :  ['./datasets/DMD/SideBody/train/c0/ga1s2sidebody_000129.png', 2, 'sad']
ga1s2sidebody_000130.png :  ['./datasets/DMD/SideBody/train/c0/ga1s2sidebody_000130.png', 0, 'neutral']
ga1s2sidebody_000131.png :  ['./datasets/DMD/SideBody/train/c0/ga1s2sidebody_000131.png', 0, 'neutral']
ga1s2sidebody_000132.png :  ['./datasets/DMD/SideBody/train/c0/ga1s2sidebody_000132.png', 0, 'neutral']
ga1s2sidebody_000133.png :  ['./datasets/DMD/SideBody/train/c0/ga1s2sidebody_000133.png', 0, 'neutral']
ga1s2sidebody_000181.png :  ['./datasets/DMD/SideBody/train/c0/ga1s2sidebody_000181.png', 1, 'happy']
ga1s2sidebody_000182.png :  ['./datasets/DMD/SideBody/train/c0/ga1s2sidebo

ga1s3sidebody_000014.png :  ['./datasets/DMD/SideBody/train/c0/ga1s3sidebody_000014.png', 0, 'neutral']
ga1s3sidebody_000015.png :  ['./datasets/DMD/SideBody/train/c0/ga1s3sidebody_000015.png', 0, 'neutral']
ga1s3sidebody_000016.png :  ['./datasets/DMD/SideBody/train/c0/ga1s3sidebody_000016.png', 0, 'neutral']
ga1s3sidebody_000017.png :  ['./datasets/DMD/SideBody/train/c0/ga1s3sidebody_000017.png', 0, 'neutral']
ga1s3sidebody_000018.png :  ['./datasets/DMD/SideBody/train/c0/ga1s3sidebody_000018.png', 0, 'neutral']
ga1s3sidebody_000019.png :  ['./datasets/DMD/SideBody/train/c0/ga1s3sidebody_000019.png', 0, 'neutral']
ga1s3sidebody_000020.png :  ['./datasets/DMD/SideBody/train/c0/ga1s3sidebody_000020.png', 0, 'neutral']
ga1s3sidebody_000021.png :  ['./datasets/DMD/SideBody/train/c0/ga1s3sidebody_000021.png', 0, 'neutral']
ga1s3sidebody_000022.png :  ['./datasets/DMD/SideBody/train/c0/ga1s3sidebody_000022.png', 0, 'neutral']
ga1s3sidebody_000023.png :  ['./datasets/DMD/SideBody/train/c0/g

ga1s4sidebody_000142.png :  ['./datasets/DMD/SideBody/train/c0/ga1s4sidebody_000142.png', 0, 'neutral']
ga1s4sidebody_000143.png :  ['./datasets/DMD/SideBody/train/c0/ga1s4sidebody_000143.png', 0, 'neutral']
ga1s4sidebody_000144.png :  ['./datasets/DMD/SideBody/train/c0/ga1s4sidebody_000144.png', 2, 'sad']
ga1s4sidebody_000145.png :  ['./datasets/DMD/SideBody/train/c0/ga1s4sidebody_000145.png', 2, 'sad']
ga1s4sidebody_000146.png :  ['./datasets/DMD/SideBody/train/c0/ga1s4sidebody_000146.png', 2, 'sad']
ga1s4sidebody_000147.png :  ['./datasets/DMD/SideBody/train/c0/ga1s4sidebody_000147.png', 0, 'neutral']
ga1s4sidebody_000148.png :  ['./datasets/DMD/SideBody/train/c0/ga1s4sidebody_000148.png', 0, 'neutral']
ga1s4sidebody_000149.png :  ['./datasets/DMD/SideBody/train/c0/ga1s4sidebody_000149.png', 0, 'neutral']
ga1s4sidebody_000150.png :  ['./datasets/DMD/SideBody/train/c0/ga1s4sidebody_000150.png', 0, 'neutral']
ga1s4sidebody_000151.png :  ['./datasets/DMD/SideBody/train/c0/ga1s4sidebody

...

gb7s4sidebody_000354.png :  ['./datasets/DMD/SideBody/train/c7/gb7s4sidebody_000354.png', 1, 'happy']
gb7s4sidebody_000362.png :  ['./datasets/DMD/SideBody/train/c7/gb7s4sidebody_000362.png', 1, 'happy']
gb7s4sidebody_000379.png :  ['./datasets/DMD/SideBody/train/c7/gb7s4sidebody_000379.png', 0, 'neutral']
gb7s4sidebody_000380.png :  ['./datasets/DMD/SideBody/train/c7/gb7s4sidebody_000380.png', 0, 'neutral']
gb7s4sidebody_000381.png :  ['./datasets/DMD/SideBody/train/c7/gb7s4sidebody_000381.png', 0, 'neutral']
gb7s4sidebody_000391.png :  ['./datasets/DMD/SideBody/train/c7/gb7s4sidebody_000391.png', 2, 'sad']
gb7s4sidebody_000405.png :  ['./datasets/DMD/SideBody/train/c7/gb7s4sidebody_000405.png', 1, 'happy']
gb7s4sidebody_000406.png :  ['./datasets/DMD/SideBody/train/c7/gb7s4sidebody_000406.png', 0, 'neutral']
gb7s4sidebody_000442.png :  ['./datasets/DMD/SideBody/train/c7/gb7s4sidebody_000442.png', 2, 'sad']
gb7s4sidebody_000443.png :  ['./datasets/DMD/SideBody/train/c7/gb7s4sidebody_0

ga1s4sidebody_000499.png :  ['./datasets/DMD/SideBody/train/c8/ga1s4sidebody_000499.png', 0, 'neutral']
ga1s4sidebody_000500.png :  ['./datasets/DMD/SideBody/train/c8/ga1s4sidebody_000500.png', 2, 'sad']
ga1s4sidebody_000501.png :  ['./datasets/DMD/SideBody/train/c8/ga1s4sidebody_000501.png', 2, 'sad']
ga1s4sidebody_000502.png :  ['./datasets/DMD/SideBody/train/c8/ga1s4sidebody_000502.png', 1, 'happy']
ga1s4sidebody_000503.png :  ['./datasets/DMD/SideBody/train/c8/ga1s4sidebody_000503.png', 2, 'sad']
ga1s4sidebody_000504.png :  ['./datasets/DMD/SideBody/train/c8/ga1s4sidebody_000504.png', 0, 'neutral']
ga1s4sidebody_000505.png :  ['./datasets/DMD/SideBody/train/c8/ga1s4sidebody_000505.png', 2, 'sad']
ga1s4sidebody_000506.png :  ['./datasets/DMD/SideBody/train/c8/ga1s4sidebody_000506.png', 2, 'sad']
ga1s4sidebody_000507.png :  ['./datasets/DMD/SideBody/train/c8/ga1s4sidebody_000507.png', 6, 'angry']
ga1s4sidebody_000508.png :  ['./datasets/DMD/SideBody/train/c8/ga1s4sidebody_000508.png'

gb6s2sidebody_000033.png :  ['./datasets/DMD/SideBody/train/c8/gb6s2sidebody_000033.png', 2, 'sad']
gb6s2sidebody_000034.png :  ['./datasets/DMD/SideBody/train/c8/gb6s2sidebody_000034.png', 1, 'happy']
gb6s2sidebody_000035.png :  ['./datasets/DMD/SideBody/train/c8/gb6s2sidebody_000035.png', 2, 'sad']
gb6s2sidebody_000036.png :  ['./datasets/DMD/SideBody/train/c8/gb6s2sidebody_000036.png', 0, 'neutral']
gb6s2sidebody_000037.png :  ['./datasets/DMD/SideBody/train/c8/gb6s2sidebody_000037.png', 2, 'sad']
gb6s2sidebody_000251.png :  ['./datasets/DMD/SideBody/train/c8/gb6s2sidebody_000251.png', 2, 'sad']
gb6s2sidebody_000252.png :  ['./datasets/DMD/SideBody/train/c8/gb6s2sidebody_000252.png', 0, 'neutral']
gb6s2sidebody_000253.png :  ['./datasets/DMD/SideBody/train/c8/gb6s2sidebody_000253.png', 0, 'neutral']
gb6s2sidebody_000254.png :  ['./datasets/DMD/SideBody/train/c8/gb6s2sidebody_000254.png', 4, 'fear']
gb6s2sidebody_000257.png :  ['./datasets/DMD/SideBody/train/c8/gb6s2sidebody_000257.p

gb7s4sidebody_000021.png :  ['./datasets/DMD/SideBody/train/c8/gb7s4sidebody_000021.png', 6, 'angry']
gb7s4sidebody_000022.png :  ['./datasets/DMD/SideBody/train/c8/gb7s4sidebody_000022.png', 1, 'happy']
gb7s4sidebody_000023.png :  ['./datasets/DMD/SideBody/train/c8/gb7s4sidebody_000023.png', 0, 'neutral']
gb7s4sidebody_000024.png :  ['./datasets/DMD/SideBody/train/c8/gb7s4sidebody_000024.png', 0, 'neutral']
gb7s4sidebody_000025.png :  ['./datasets/DMD/SideBody/train/c8/gb7s4sidebody_000025.png', 0, 'neutral']
gb7s4sidebody_000026.png :  ['./datasets/DMD/SideBody/train/c8/gb7s4sidebody_000026.png', 1, 'happy']
gb7s4sidebody_000027.png :  ['./datasets/DMD/SideBody/train/c8/gb7s4sidebody_000027.png', 0, 'neutral']
gb7s4sidebody_000028.png :  ['./datasets/DMD/SideBody/train/c8/gb7s4sidebody_000028.png', 1, 'happy']
gb7s4sidebody_000029.png :  ['./datasets/DMD/SideBody/train/c8/gb7s4sidebody_000029.png', 1, 'happy']
gb7s4sidebody_000242.png :  ['./datasets/DMD/SideBody/train/c8/gb7s4sidebo

gb6s1sidebody_000190.png :  ['./datasets/DMD/SideBody/train/c9/gb6s1sidebody_000190.png', 1, 'happy']
gb6s1sidebody_000192.png :  ['./datasets/DMD/SideBody/train/c9/gb6s1sidebody_000192.png', 1, 'happy']
gb6s3sidebody_000003.png :  ['./datasets/DMD/SideBody/train/c9/gb6s3sidebody_000003.png', 1, 'happy']
gb6s3sidebody_000004.png :  ['./datasets/DMD/SideBody/train/c9/gb6s3sidebody_000004.png', 1, 'happy']
gb7s1sidebody_000071.png :  ['./datasets/DMD/SideBody/train/c9/gb7s1sidebody_000071.png', 1, 'happy']
gb7s1sidebody_000073.png :  ['./datasets/DMD/SideBody/train/c9/gb7s1sidebody_000073.png', 1, 'happy']
gb7s1sidebody_000074.png :  ['./datasets/DMD/SideBody/train/c9/gb7s1sidebody_000074.png', 1, 'happy']
gb7s1sidebody_000075.png :  ['./datasets/DMD/SideBody/train/c9/gb7s1sidebody_000075.png', 1, 'happy']
gb7s1sidebody_000089.png :  ['./datasets/DMD/SideBody/train/c9/gb7s1sidebody_000089.png', 1, 'happy']
gb7s1sidebody_000160.png :  ['./datasets/DMD/SideBody/train/c9/gb7s1sidebody_00016